# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902015


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:02<00:41,  1.41s/it]

Rendering models:  10%|▉         | 3/31 [00:02<00:29,  1.04s/it]

Rendering models:  16%|█▌        | 5/31 [00:03<00:19,  1.32it/s]

Rendering models:  19%|█▉        | 6/31 [00:04<00:25,  1.01s/it]

...

Rendering models:  77%|███████▋  | 24/31 [00:10<00:01,  5.56it/s]

Rendering models:  87%|████████▋ | 27/31 [00:10<00:00,  6.28it/s]

Rendering models:  90%|█████████ | 28/31 [00:10<00:00,  6.73it/s]

Rendering models:  94%|█████████▎| 29/31 [00:11<00:00,  5.60it/s]

equidad1                              0.000981
not-logged-in-27a50043cebc294af948    0.000517
accurato                              0.003023
Tommy80452                            0.000937
MEMcKinney                            0.002321
kayleebug2017                         0.002505
not-logged-in-aac1a549b28981db19c0    0.004565
JessieDawn                            0.002307
ElisabethB                            0.000638
readsandwiches                        0.109790
elizabeth_s                           0.004145
not-logged-in-3a93234a2311f6df1afd    0.001148
kawaboy                               0.003432
not-logged-in-5f30b2bb9b7f9d49da41    4.727864
sepsamadi                             0.001940
LunaMoona                             0.001531
Planetme                              0.001356
not-logged-in-214814e52004b42491a4    0.002642
pangeli5                              0.000514
rhondacard                            0.004941
Lavadude                              0.020110
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())